In [1]:
import pandas as pd
import joblib

In [2]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
posters = pd.read_csv('movie_posters.csv',header=None,names=['movieId','poster_url'],index_col=0)
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
ratings.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [3]:
user_ratings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
user_ratings.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
## Let's Remove Movies which have less than 10 users who rated it. and fill remaining NaN with 0
user_ratings = user_ratings.dropna(thresh=10,axis=1).fillna(0)
user_ratings.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
## Let's Build our Similarity Matrix
item_similarity_df = user_ratings.corr(method='pearson')
item_similarity_df.head(50)

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,0.063117,-0.023768,0.143482,0.011998,0.087931,0.224052,0.034223,0.009277,0.008331,...,0.017477,0.032470,0.134701,0.153158,0.101301,0.049897,0.003233,0.187953,0.062174,0.353194
(500) Days of Summer (2009),0.063117,1.000000,0.142471,0.273989,0.193960,0.148903,0.142141,0.159756,0.135486,0.200135,...,0.374515,0.178655,0.068407,0.414585,0.355723,0.252226,0.216007,0.053614,0.241092,0.125905
10 Cloverfield Lane (2016),-0.023768,0.142471,1.000000,-0.005799,0.112396,0.006139,-0.016835,0.031704,-0.024275,0.272943,...,0.242663,0.099059,-0.023477,0.272347,0.241751,0.195054,0.319371,0.177846,0.096638,0.002733
10 Things I Hate About You (1999),0.143482,0.273989,-0.005799,1.000000,0.244670,0.223481,0.211473,0.011784,0.091964,0.043383,...,0.243118,0.104858,0.132460,0.091853,0.158637,0.281934,0.050031,0.121029,0.130813,0.110612
"10,000 BC (2008)",0.011998,0.193960,0.112396,0.244670,1.000000,0.234459,0.119132,0.059187,-0.025882,0.089328,...,0.260261,0.087592,0.094913,0.184521,0.242299,0.240231,0.094773,0.088045,0.203002,0.083518
101 Dalmatians (1996),0.087931,0.148903,0.006139,0.223481,0.234459,1.000000,0.285112,0.119843,0.072399,0.029967,...,0.114968,0.077232,0.096294,0.067134,0.113224,0.184324,0.054024,0.047804,0.156932,0.078734
101 Dalmatians (One Hundred and One Dalmatians) (1961),0.224052,0.142141,-0.016835,0.211473,0.119132,0.285112,1.000000,0.134037,0.017264,-0.046277,...,0.120302,0.125816,0.049818,0.083650,0.171654,0.274260,0.077594,0.085606,0.248820,0.171118
12 Angry Men (1957),0.034223,0.159756,0.031704,0.011784,0.059187,0.119843,0.134037,1.000000,0.132979,0.058862,...,0.104518,0.028415,0.079905,0.241435,0.144652,0.122107,0.056742,-0.001708,0.074306,0.102744
12 Years a Slave (2013),0.009277,0.135486,-0.024275,0.091964,-0.025882,0.072399,0.017264,0.132979,1.000000,0.249931,...,0.024045,0.038127,0.013786,0.190366,0.104150,0.017351,0.063325,0.002528,0.037469,0.004213


In [8]:
joblib.dump(item_similarity_df,"item_similarity_df")

['item_similarity_df']

In [6]:
def get_similar_movies(movie_name,user_rating):
    similar_score = item_similarity_df[movie_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    
    return similar_score

In [28]:
action_lover = [("Catch Me If You Can (2002)",5),
                ("12 Years a Slave (2013)",3),("2012 (2009)",3),
               ("(500) Days of Summer (2009)",5),("28 Days Later (2002)",4),("Deadpool 2 (2018)",4),("Blood Diamond (2006)",4),
                ("Up in the Air (2009)",4),("I Am Legend (2007)",5),("Titanic (1997)",5),("Up (2009)",4),("Kung Fu Panda (2008)",4),
               ("I Am Legend (2007)",5),("Sherlock Holmes (2009)",4),("Da Vinci Code, The (2006)",4),("Casino Royale (2006)",4),
                ("300 (2007)",2),("War of the Worlds (2005)",2),("Scott Pilgrim vs. the World (2010)",3),("Casino Royale (2006)",4),
                ("Prestige, The (2006)",4),("Wrestler, The (2008)",4),("Mad Max: Fury Road (2015)",5),("King Kong (2005)",5),
                ("Minority Report (2002)",4),("Edge of Tomorrow (2014)",4),("Django Unchained (2012)",3),("Troy (2004)",3),
                ("Star Trek (2009)",4),("Kung Fu Panda (2008)",4),("Black Hawk Down (2001)",4),("Social Network, The (2010)",4),
                ("V for Vendetta (2006)",2)
               
               ]

def check_seen(movie,seen_movies):
    for item,rating in seen_movies:
        if item == movie:
            return True
    
    return False
        
    
romantic_lover = [("Titanic (1997)",5),("Up (2009)",4),("Up (2009)",4),("Kung Fu Panda (2008)",4)]

similar_movies = pd.DataFrame()

for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar_movies(movie,rating),ignore_index=True)

all_recommend = similar_movies.sum().sort_values(ascending=False)

for movie,score in all_recommend.iteritems():
    if not check_seen(movie,action_lover):
        print(movie)

Hangover, The (2009)
Avatar (2009)
Iron Man (2008)
Star Wars: Episode III - Revenge of the Sith (2005)
District 9 (2009)
Zombieland (2009)
Batman Begins (2005)
Day After Tomorrow, The (2004)
Kick-Ass (2010)
I, Robot (2004)
WALL·E (2008)
Pirates of the Caribbean: Dead Man's Chest (2006)
Quantum of Solace (2008)
Wedding Crashers (2005)
X-Men Origins: Wolverine (2009)
Zodiac (2007)
Sin City (2005)
Dark Knight, The (2008)
Incredible Hulk, The (2008)
X-Men: First Class (2011)
Incredibles, The (2004)
Fantastic Four (2005)
Taken (2008)
Hurt Locker, The (2008)
Toy Story 3 (2010)
Hot Fuzz (2007)
Hellboy (2004)
Kill Bill: Vol. 1 (2003)
Ocean's Eleven (2001)
Matrix Revolutions, The (2003)
Pirates of the Caribbean: At World's End (2007)
Watchmen (2009)
3:10 to Yuma (2007)
Pirates of the Caribbean: The Curse of the Black Pearl (2003)
Matrix Reloaded, The (2003)
Captain America: The First Avenger (2011)
Inception (2010)
Hancock (2008)
Last Samurai, The (2003)
Simpsons Movie, The (2007)
The Hunger Ga

Kung Fu Hustle (Gong fu) (2004)
Monty Python and the Holy Grail (1975)
Terminator, The (1984)
Cinderella Man (2005)
Prisoners (2013)
Rain Man (1988)
Mean Girls (2004)
Whiplash (2014)
10,000 BC (2008)
Futurama: Bender's Big Score (2007)
Time Machine, The (2002)
RoboCop (1987)
Pianist, The (2002)
Joe Dirt (2001)
O Brother, Where Art Thou? (2000)
Atlantis: The Lost Empire (2001)
Captain Phillips (2013)
Spy (2015)
Eastern Promises (2007)
Dungeons & Dragons (2000)
Zero Dark Thirty (2012)
Rocky II (1979)
Fistful of Dollars, A (Per un pugno di dollari) (1964)
Mask of Zorro, The (1998)
Nick and Norah's Infinite Playlist (2008)
Princess Mononoke (Mononoke-hime) (1997)
Secondhand Lions (2003)
Secretary (2002)
The Jungle Book (2016)
Anchorman 2: The Legend Continues (2013)
Warm Bodies (2013)
Star Wars: Episode IV - A New Hope (1977)
Legally Blonde (2001)
Predestination (2014)
Dictator, The (2012)
Road Trip (2000)
Walk the Line (2005)
Get Him to the Greek (2010)
Peter Pan (1953)
Transformers: Dark

Drop Dead Gorgeous (1999)
Look Who's Talking (1989)
My Best Friend's Wedding (1997)
Get Out (2017)
Death to Smoochy (2002)
Sixteen Candles (1984)
Grumpy Old Men (1993)
Last Action Hero (1993)
WarGames (1983)
Mona Lisa Smile (2003)
Muppet Treasure Island (1996)
Save the Last Dance (2001)
Any Given Sunday (1999)
Color of Money, The (1986)
Fried Green Tomatoes (1991)
Meet the Robinsons (2007)
Maleficent (2014)
JFK (1991)
Eyes Wide Shut (1999)
Evil Dead II (Dead by Dawn) (1987)
Horrible Bosses 2 (2014)
Rushmore (1998)
River Runs Through It, A (1992)
Rescuers Down Under, The (1990)
What Lies Beneath (2000)
Twin Peaks: Fire Walk with Me (1992)
She's All That (1999)
Adaptation (2002)
Science of Sleep, The (La science des rêves) (2006)
Red Planet (2000)
Gremlins 2: The New Batch (1990)
Adventures in Babysitting (1987)
All the President's Men (1976)
Dirty Dancing (1987)
Awakenings (1990)
Date Night (2010)
My Cousin Vinny (1992)
America's Sweethearts (2001)
Lake Placid (1999)
Hoosiers (a.k.a. Be

Young Guns II (1990)
Apollo 13 (1995)
Some Kind of Wonderful (1987)
Thirteenth Floor, The (1999)
State and Main (2000)
Red Dawn (1984)
Made in America (1993)
Cinema Paradiso (Nuovo cinema Paradiso) (1989)
Lord of the Flies (1963)
Multiplicity (1996)
Thirteen Days (2000)
Jungle2Jungle (a.k.a. Jungle 2 Jungle) (1997)
Rebecca (1940)
Drugstore Cowboy (1989)
Lake House, The (2006)
Desperately Seeking Susan (1985)
Son in Law (1993)
Opposite of Sex, The (1998)
Mosquito Coast, The (1986)
Glory (1989)
Excalibur (1981)
Shine (1996)
Soylent Green (1973)
On Golden Pond (1981)
Sleepers (1996)
Aristocrats, The (2005)
High Plains Drifter (1973)
Big Night (1996)
Dead Again (1991)
King Kong (1976)
Boot, Das (Boat, The) (1981)
Summer of Sam (1999)
It's a Mad, Mad, Mad, Mad World (1963)
Menace II Society (1993)
Fugitive, The (1993)
Child's Play (1988)
Doors, The (1991)
Airplane II: The Sequel (1982)
Doctor Zhivago (1965)
Man Who Would Be King, The (1975)
All Dogs Go to Heaven (1989)
Big Blue, The (Grand 

In [8]:
# # sim_movies.merge(movies)

# from IPython.display import Image
# from IPython.display import display
# for movie,score in sim_movies.iteritems():
    
#     movie_id = movies[movies['title']==movie]['movieId'].values[0]
#     print(movie_id)
#     try:
#         print(posters.loc[movie_id])
#     except:
#         pass
# posters.loc[1721]    

In [9]:
#for movie,score in sim_movies.iteritems(): 

# toy_story = 'https://images-na.ssl-images-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@..jpg'
# Image(url=toy_story,width=100, height=100)
